In [1]:
import os
import pandas as pd
import json

def parse_data(filename):
    df1 = pd.read_csv(filename)
    df2 = df1.rename(columns={"trnOpDate":"乘車日",
            "staCode":"車站代碼",
            "gateInComingCnt":"進站人數",
            "gateOutGoingCnt":"出站人數"})
    with open('車站基本資料集.json',encoding='utf-8') as file:
        json_data = json.load(file)
        stations_info = pd.DataFrame(json_data,columns=['stationCode','stationName'])
        stations_info = stations_info.astype({'stationCode':'int32'})
    df3 = pd.merge(df2,stations_info,left_on='車站代碼',right_on='stationCode')
    df4 = df3.rename(columns={'stationName':'車站名稱'})
    df5 = df4.reindex(columns=['乘車日','車站名稱','進站人數','出站人數'])
    df5['乘車日'] = pd.to_datetime(df5['乘車日'].astype(str))
    return df5

def main():
    current_dir = os.getcwd()
    csv_dir = os.path.join(current_dir,'每日各站進出站人數')
    files_and_dirs = os.listdir(csv_dir)
    all_csv_files = [filename for filename in files_and_dirs if '每日各站進出站人數' in filename]
    sorted_csv_files = sorted(all_csv_files)
    all_final_dfs = [] 
    for filename in sorted_csv_files:
        abs_filename = os.path.join(csv_dir,filename)
        all_final_dfs.append(parse_data(abs_filename))
    df_done = pd.concat(all_final_dfs)
    df_done1 = df_done.set_index('乘車日')
    return df_done1 


if __name__ == '__main__':
    df_done1 = main()

    

In [2]:
df_done1

,車站名稱,進站人數,出站人數
乘車日,,,
2019-04-23,基隆,8442,7743
2019-04-23,三坑,1394,1348
2019-04-23,八堵,2770,2423
2019-04-23,七堵,6113,6335
2019-04-23,百福,2680,2726
...,...,...,...
2023-12-31,瑞芳,7916,8252
2023-12-31,海科館,164,195
2023-12-31,八斗子,652,720


In [17]:
years = df_done1.index.to_series().dt.year.values
years


array([2019, 2019, 2019, ..., 2023, 2023, 2023], dtype=int32)

In [22]:
groupby1 = df_done1.groupby(by=years)
len(groupby1)
groupby1[['進站人數','出站人數']].sum()

,進站人數,出站人數
2019,166359521,166359521
2020,208280011,208280011
2021,159324537,159324537
2022,175359623,175359623
2023,224530482,224530482


In [25]:
months = df_done1.index.to_series().dt.month.values
months

array([ 4,  4,  4, ..., 12, 12, 12], dtype=int32)

In [27]:
groupby2 = df_done1.groupby(by=[years, months])
len(groupby2)
groupby2[['進站人數','出站人數']].sum()

進站人數      出站人數
2019 4    5170399   5170399
     5   19534503  19534503
     6   19577044  19577044
     7   20030860  20030860
     8   19775282  19775282
     9   19622555  19622555
     10  20983360  20983360
     11  20476773  20476773
     12  21188745  21188745
2020 1   19993675  19993675
     2   15671872  15671872
     3   15315217  15315217
     4   13113386  13113386
     5   15326842  15326842
     6   16976704  16976704
     7   18298257  18298257
     8   18518554  18518554
     9   17698221  17698221
     10  19477695  19477695
     11  18711859  18711859
     12  19177729  19177729
2021 1   16800677  16800677
     2   15856043  15856043
     3   18343871  18343871
     4   18556362  18556362
     5   10759039  10759039
     6    3718089   3718089
     7    5205111   5205111
     8    9338534   9338534
     9   11893156  11893156
     10  15127245  15127245
     11  16010305  16010305
     12  17716105  17716105
2022 1   14331609  14331609
     2   13090244  13090244
     3   16465704  16465704
     4   13741557  13741557
     5    9699057   9699057
     6   10838883  10838883
     7   14225428  14225428
     8   15660748  15660748
     9   15458656  15458656
     10  16528375  16528375
     11  17161542  17161542
     12  18157820  18157820
2023 1   17787014  17787014
     2   17156824  17156824
     3   18346962  18346962
     4   18753172  18753172
     5   18195079  18195079
     6   17520338  17520338
     7   17888496  17888496
     8   18504559  18504559
     9   18682571  18682571
     10  20303009  20303009
     11  20023786  20023786
     12  21368672  21368672